In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re # regex
from datetime import datetime

In [ ]:
## visualize rise and cease of lego parts over time in a density plot each part_num
# note that processing takes forever (50 min) depending on number of entries you want

# load required datasets from csv
inv_parts = pd.read_csv("../data/raw/inventory_parts.csv")
inv_fig = pd.read_csv("../data/raw/inventory_minifigs.csv")
parts = pd.read_csv("../data/raw/parts.csv")
sets = pd.read_csv("../data/raw/sets.csv")
inventories = pd.read_csv("../data/raw/inventories.csv")
relation = pd.read_csv("../data/raw/part_relationships.csv")

# begin logging steps and time
log = []
log.append([0, "begin processing", datetime.now().strftime("%H:%M:%S")])
print("begin processing, time: ", datetime.now().strftime("%H:%M:%S"))

# drop sets with less than 3 parts and themes which are not really lego, like 'gear' whichs contains backpacks and mugs, or service packs for spare parts
themesToDrop = [443, 524, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 685, 518, 258, 519, 520, 521, 259, 261, 260, 262, 500, 501, 502, 503, 683, 598, 497, 498, 499, 480, 213]
# gather set_num according to theme_id and num_parts < 3
setsToDrop = []
for t in themesToDrop:
    setsToDrop += sets[sets['theme_id'] == t].set_num.to_list()
setsToDrop += sets[sets['num_parts'] < 3].set_num.to_list()
setsToDrop = list(dict.fromkeys(setsToDrop)) 
# drop set_num in sets data
setsToDropToId = []
for s in setsToDrop:
    setsToDropToId.append(sets[sets['set_num'] == s].index.values[0])
sets.drop(setsToDropToId, inplace=True)
# drop inventory ids according to set_num in inv_parts data and inv_fig data
setsToDropToInvId = []
for s in setsToDrop:
    setsToDropToInvId.extend(inventories[inventories['set_num'] == s].id.to_list())
setsToDropToInvIdToId = []
for i in setsToDropToInvId:
    setsToDropToInvIdToId.extend(inv_parts[inv_parts['inventory_id'] == i].index.to_list())
inv_parts.drop(setsToDropToInvIdToId, inplace=True)

setsToDropToInvIdToId = []
for i in setsToDropToInvId:
    setsToDropToInvIdToId.extend(inv_fig[inv_fig['inventory_id'] == i].index.to_list())
inv_fig.drop(setsToDropToInvIdToId, inplace=True)

log.append([0, "after remove themes and very small sets", datetime.now().strftime("%H:%M:%S")])
print("after remove themes and very small sets, time: ", datetime.now().strftime("%H:%M:%S"))

# replace child_part_num with parent_part_num in inventories df (multiply times since parents can be childs too)
invParent = inv_parts.copy()
childToParent = pd.Series(relation.parent_part_num.values, index=relation.child_part_num).to_dict()

invParent['part_num'].replace(childToParent, inplace=True)
log.append([invParent.part_num.nunique(), "after childToParent 1", datetime.now().strftime("%H:%M:%S")])
print(invParent.part_num.nunique(), " after childToParent 1, time: ", datetime.now().strftime("%H:%M:%S"))

invParent['part_num'].replace(childToParent, inplace=True)
log.append([invParent.part_num.nunique(), "after childToParent 2", datetime.now().strftime("%H:%M:%S")])
print(invParent.part_num.nunique(), " after childToParent 2, time: ", datetime.now().strftime("%H:%M:%S"))

invParent['part_num'].replace(childToParent, inplace=True)
log.append([invParent.part_num.nunique(), "after childToParent 3", datetime.now().strftime("%H:%M:%S")])
print(invParent.part_num.nunique(), " after childToParent 3, time: ", datetime.now().strftime("%H:%M:%S"))

invParent['part_num'].replace(childToParent, inplace=True)
log.append([invParent.part_num.nunique(), "after childToParent 4", datetime.now().strftime("%H:%M:%S")])
print(invParent.part_num.nunique(), " after childToParent 4, time: ", datetime.now().strftime("%H:%M:%S"))

# eliminate all part_num containing stickers, non-lego (eg booklets), printed or pattern or mold or alternate versions st only parents remain in partsClean.part_num
partsClean = parts.copy()
childPartNumToId = []
for p in relation.child_part_num:
    childPartNumToId.append(partsClean[partsClean['part_num'] == p].index.values[0])
partsClean.drop(childPartNumToId, inplace=True)
partsClean.drop(partsClean[partsClean['part_cat_id'] == 58].index, inplace=True) # drop Stickers
partsClean.drop(partsClean[partsClean['part_cat_id'] == 17].index, inplace=True) # drop Non-Lego (eg booklets)
partsClean.drop(partsClean[partsClean['part_cat_id'] == 4].index, inplace=True) # drop Duplo, Quatro and Primo
partsClean.drop(partsClean[partsClean['part_cat_id'] == 43].index, inplace=True) # drop Znap
partsClean.drop(partsClean[partsClean['part_cat_id'] == 48].index, inplace=True) # drop Clikits
partsClean.drop(partsClean[partsClean['part_cat_id'] == 57].index, inplace=True) # drop Non-Buildable Figures 
#partsClean.drop(partsClean[partsClean['part_cat_id'] == 42].index, inplace=True) # drop Belville, Scala, Fabuland
#partsClean.drop(partsClean[partsClean['part_cat_id'] == 41].index, inplace=True) # drop Bionicle, Hero factory, Constraction
partsClean.drop(partsClean[partsClean['part_cat_id'] == 66].index, inplace=True) # drop Modulex
partsClean.drop(partsClean[partsClean['part_cat_id'] == 45].index, inplace=True) # drop Mindstorms and Electric
#partsClean.drop(partsClean[partsClean['part_cat_id'] == 44].index, inplace=True) # drop Mechanical eg wind up motors

log.append([0, "after partsClean", datetime.now().strftime("%H:%M:%S")])
print("after partsClean, time: ", datetime.now().strftime("%H:%M:%S"))

# now replace remaining "<1>pr<2>" with "<1>" and drop duplicated "<1>" entries to eliminate remaining prints, necessary because they were not dropped above due to no relationship by missing parent (so we make one up as "<1>")
partsToRemove = {}
for p in partsClean.part_num:
    pSub = re.sub("(\S+[0-9]+)([a-zA-Z]+\S+)", lambda pat: pat.group(1), p)
    pSub = re.sub("(\S+[0-9]+)([a-zA-Z]+\S+)", lambda pat: pat.group(1), pSub)
    pSub = re.sub("(\S+[0-9]+)([a-zA-Z]+\S+)", lambda pat: pat.group(1), pSub)
    pSub = re.sub("(\S+[0-9]+)([a-zA-Z]+\S+)", lambda pat: pat.group(1), pSub)
    pSub = re.sub("(\S+[0-9]+)([a-zA-Z]+)", lambda pat: pat.group(1), pSub)
    partsToRemove.update({p : pSub})

partsClean['part_num'].replace(partsToRemove, inplace=True)
partsClean.drop_duplicates(subset='part_num', keep='first', inplace=True)

invParent['part_num'].replace(partsToRemove, inplace=True)

log.append([0, "after partsToRemove", datetime.now().strftime("%H:%M:%S")])
print("after partsToRemove, time: ", datetime.now().strftime("%H:%M:%S"))

# map the sets which contain part_num to the according year
partsOverTime_sets = pd.DataFrame(columns=['part_num', 'name']+list(range(1949,2021)))
# loop over all part_num p in inv_parts.part_num to fill years of set occurences in df partsOverTime_sets 
for p in partsClean.part_num:
    # idk if good or bad coding style, but it makes me feel safe and helps debugging
    partToId = 0
    idToSet = 0
    setToYear = 0
    figToId = 0
    removeFig = []
    # init part dict with part_num, name and years from 1949 to 2020
    part = {'part_num' : p}
    part.update({'name' : partsClean[partsClean['part_num'] == p].iloc[0,1]})
    for y in list(range(1949, 2021)):
        part.update({y: 0})
    # part to inventory id (list)
    partToId = invParent[invParent['part_num'] == p].inventory_id.to_list()
    # remove duplicates from list, since these are parts occuring multiple times due different colors
    partToId = list(dict.fromkeys(partToId)) 
    # inventory id to set num
    idToSet = [inventories[inventories['id'] == e].iloc[0,2] for e in partToId]
    # in case of figures (set_num = fig-...) we need to go to inv_fig to get inv_id (list) -> inventories to get set_num(s) 
    for f in idToSet:
        if re.match(r"fig-[0-9]+", f):
            figToId = inv_fig[inv_fig['fig_num'] == f].inventory_id.to_list()
            removeFig.append(f)
            idToSet.extend([inventories[inventories['id'] == e].iloc[0,2] for e in figToId])
    idToSet = [e for e in idToSet if e not in removeFig]
    # drop all duplicetd set_num since they came from versions >1 of inventories, otherwise sets would occur more then once per part 
    idToSet = list(dict.fromkeys(idToSet)) 
    # set num to year
    setToYear = [sets[sets['set_num'] == e].iloc[0,2] for e in idToSet]
    # update dict of part with occurences per year
    for e in setToYear:
            part[e] += 1
    # finally append dict part as row to df partsOverTime_sets
    partsOverTime_sets = partsOverTime_sets.append(part, ignore_index=True)

    log.append([p, "after fill years in partsOverTime_sets", datetime.now().strftime("%H:%M:%S")])
    print(p, ", after fill years in partsOverTime_sets, time: ", datetime.now().strftime("%H:%M:%S"))

# sum up all occurences in as set of a part, add it in a new column and drop all parts which never occur in any set
partsOverTime_sets.insert(2, 'sum', partsOverTime_sets[list(range(1949,2021))].sum(axis=1))
partsOverTime_sets.drop(partsOverTime_sets[partsOverTime_sets['sum'] == 0].index, inplace=True)

# add a row with number of different parts occuring in a year
partsPerYear = (partsOverTime_sets != 0).astype(int).sum(axis=0).to_dict()
partsPerYear.update({'name': 'number of parts'})
partsOverTime_sets = pd.concat([pd.DataFrame(partsPerYear, index=[0]),partsOverTime_sets]).reset_index(drop=True)

# add a row with number of sets which are released this year
setsPerYear = sets.pivot_table(index=['year'], aggfunc='size').to_dict()
setsPerYear.update({'name': 'number of sets'})
setsPerYear = pd.DataFrame(columns=['part_num', 'name', 'sum']+list(range(1949,2021))).append(setsPerYear, ignore_index=True).fillna(0)
partsOverTime_sets = pd.concat([setsPerYear, partsOverTime_sets]).reset_index(drop=True)

log.append([0, "after some more features", datetime.now().strftime("%H:%M:%S")])
print("after some more features, time: ", datetime.now().strftime("%H:%M:%S"))

# create csv with sets(years) series: part_num, name, year_<min>, ..., year_<max> <- which contain the quantity of sets with part_num in this year
# note that min(year) = 1949 and max(year) = 2020 but only until may
partsOverTime_sets.to_csv("../data/processed/partsOverTime.csv", index = False)

log.append([0, "after write csv", datetime.now().strftime("%H:%M:%S")])
print("after write csv, time: ", datetime.now().strftime("%H:%M:%S"))

## maybe later:
# map part quantity to the accoring year, note that diffrent colors shall count as same part
# create csv with quantity(years) series: part_num, name, year_<min>, ..., year_<max> <- which contain the quantity overall for part_num in this year

In [ ]:
# partsClean.part_num.to_csv("../data/processed/partsClean.part_num.csv", index = False)
# pd.DataFrame(partsToRemove.items()).to_csv("../data/processed/diff0.csv", index = False, columns=[0])
# parts.nunique()

In [ ]:
### run above cells before, in order to load and process data ###

# add a row with number of minfig sets which are released this year
# array with all themes that are exclusively for minifigs
themesMinifig = [609, 611, 618, 622, 656, 675, 677, 681, 684, 691, 692]+list(range(535,558))
# gather all set_num which are of above themes 
setsMinifig = []
for t in themesMinifig:
    setsMinifig += sets[sets['theme_id'] == t].set_num.to_list()
# create complement setsMinifig to drop all sets without minifig
setsWoMinifig = [e for e in sets.set_num.to_list() if e not in setsMinifig]
# now drop the complement set to have sets data with only minifig sets
setsWoMinifigToId = []
for s in setsWoMinifig:
    setsWoMinifigToId.append(sets[sets['set_num'] == s].index.values[0])
setsMinifigs = sets.copy().drop(setsWoMinifigToId)
# write minifig sets per year in parts over time data
minifigSetsPerYear = setsMinifigs.pivot_table(index=['year'], aggfunc='size').to_dict()
minifigSetsPerYear.update({'name': 'number of minifig sets'})
minifigSetsPerYear = pd.DataFrame(columns=['part_num', 'name', 'sum']+list(range(1949,2021))).append(minifigSetsPerYear, ignore_index=True).fillna(0)
partsOverTime_sets = pd.concat([minifigSetsPerYear, partsOverTime_sets]).reset_index(drop=True)

# write csv again
partsOverTime_sets.to_csv("../data/processed/partsOverTime.csv", index = False)